#**1. Decoding Transaction Dynamics on PhonePe**

In [1]:
import sys
import os

sys.path.append(os.path.abspath("D:/D26_Files/Phonepe_Analytics/Pulse_Case_Studies/"))

from db_connection import get_phonepe_engine


engine = get_phonepe_engine()


In [2]:
import pandas as pd
from sqlalchemy import text

In [3]:
tables_df = pd.read_sql("SHOW TABLES;", engine)
tables_df

,Tables_in_phonepe_db
0,agg_transaction
1,agg_user
2,insurance_transaction
3,map_transaction
4,map_user
5,top_transaction


In [4]:
Agg_Transaction_df = pd.read_sql("SELECT * FROM agg_transaction;", engine)
Agg_Transaction_df.head()

,State,Year,Quarter,Transaction_type,Transaction_count,Transaction_amount
0,Andaman & Nicobar Islands,2018,1,Recharge & bill payments,4200,1.845307e+06
1,Andaman & Nicobar Islands,2018,1,Peer-to-peer payments,1871,1.213866e+07
2,Andaman & Nicobar Islands,2018,1,Merchant payments,298,4.525072e+05
3,Andaman & Nicobar Islands,2018,1,Financial Services,33,1.060142e+04
4,Andaman & Nicobar Islands,2018,1,Others,256,1.846899e+05


*1.	How have total transactions (volume & value) grown over time across India?*

In [5]:
growth_df = pd.read_sql("""
SELECT 
Year,
Quarter,
    SUM(Transaction_count) AS Total_volume,
    SUM(Transaction_amount) AS Total_value
FROM agg_transaction
GROUP BY Year, Quarter
ORDER BY Year, Quarter;
""", engine)
growth_df.head()

,Year,Quarter,Total_volume,Total_value
0,2018,1,134425599.0,1.718334e+11
1,2018,2,187365440.0,3.043742e+11
2,2018,3,341299764.0,4.751015e+11
3,2018,4,417111607.0,6.717362e+11
4,2019,1,708992981.0,9.900214e+11


*2.	Which payment categories (recharge, bills, merchant payments, P2P, etc.) show the fastest growth vs stagnation?*

In [6]:
categories_df = pd.read_sql("""
SELECT 
Year,
Transaction_type,
       SUM(Transaction_count) as Total_Volume, 
       SUM(Transaction_amount) as Total_Value 
FROM agg_transaction 
GROUP BY Transaction_type, Year 
ORDER BY Transaction_type, Year;
""", engine)
categories_df.head()

,Year,Transaction_type,Total_Volume,Total_Value
0,2018,Financial Services,20384145.0,3.752877e+09
1,2019,Financial Services,12624867.0,2.317506e+09
2,2020,Financial Services,8514716.0,8.457813e+09
3,2021,Financial Services,13726318.0,1.629694e+10
4,2022,Financial Services,15890732.0,1.868558e+10


*3.	Are there seasonal spikes in transaction activity (e.g., festive quarters)?*

In [7]:
SNS_df = pd.read_sql("""
SELECT
Year, 
Quarter,
    SUM(Transaction_count) AS Total_volume,
    SUM(Transaction_amount) AS Total_value
FROM agg_transaction
GROUP BY Year, Quarter
ORDER BY Quarter, Year;""", engine)
SNS_df.head()

,Year,Quarter,Total_volume,Total_value
0,2018,1,1.344256e+08,1.718334e+11
1,2019,1,7.089930e+08,9.900214e+11
2,2020,1,1.623038e+09,2.697112e+12
3,2021,1,3.457585e+09,6.284243e+12
4,2022,1,7.593180e+09,1.323963e+13


*4.	Which states are the top contributors to overall transaction growth?*

In [8]:
Top_state_df = pd.read_sql("""
SELECT
State,
    SUM(Transaction_count) AS Total_volume,
    SUM(Transaction_amount) AS Total_value
FROM agg_transaction
GROUP BY State
ORDER BY Total_value DESC
LIMIT 10;""" , engine)
Top_state_df

,State,Total_volume,Total_value
0,Telangana,2.617468e+10,4.165596e+13
1,Karnataka,3.097095e+10,4.067872e+13
2,Maharashtra,3.198521e+10,4.037420e+13
3,Andhra Pradesh,1.891870e+10,3.466908e+13
4,Uttar Pradesh,1.852360e+10,2.688521e+13
5,Rajasthan,1.710854e+10,2.634324e+13
6,Madhya Pradesh,1.407218e+10,1.912528e+13
7,Bihar,1.094103e+10,1.790135e+13
8,West Bengal,9.191500e+09,1.558416e+13
9,Odisha,8.918527e+09,1.226398e+13


*5.	Are there states showing decline or stagnation despite a national upward trend?*

In [9]:
State_trends_df = pd.read_sql("""
SELECT 
State, 
     SUM(Transaction_count) AS Total_Volume
FROM agg_transaction
GROUP BY State
ORDER BY Total_Volume ASC
Limit 10;
""", engine)
State_trends_df

,State,Total_Volume
0,Lakshadweep,883994.0
1,Mizoram,19535750.0
2,Ladakh,39345875.0
3,Andaman & Nicobar Islands,39706951.0
4,Nagaland,54768743.0
5,Sikkim,65860842.0
6,Manipur,73350340.0
7,Meghalaya,88635419.0
8,Tripura,120838496.0
9,Arunachal Pradesh,160223357.0
